In [ ]:
import os
import glob

import numpy as np
import pandas as pd
from toolz import compose
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from scipy.stats import linregress


import sys
sys.path.append('..')

import modules.plotting as pl
import modules.signals as sig
import modules.numpy_funcs as nf
import modules.pandas_funcs as pf
import modules.gait_metrics as gm
import modules.assign_sides as asi
import modules.sliding_window as sw
import modules.iterable_funcs as itf
import modules.linear_algebra as lin
import modules.phase_detection as pde


In [ ]:
%matplotlib auto

In [ ]:
file_name = '2014-12-08_P004_Post_004'

In [ ]:
best_pos_dir = os.path.join('..', 'data', 'kinect', 'best pos')
hypo_dir = os.path.join('..', 'data', 'kinect', 'processed', 'hypothesis')

hypo_paths = glob.glob(os.path.join(hypo_dir, '*.pkl'))
best_pos_paths = glob.glob(os.path.join(best_pos_dir, '*.pkl'))

hypo_paths = [x for x in hypo_paths if file_name in x]
best_pos_paths = [x for x in best_pos_paths if file_name in x]

df_hypo = pd.read_pickle(hypo_paths[0])
df_best_pos = pd.read_pickle(best_pos_paths[0])

## Split trial into separate passes

In [ ]:
# Convert all position vectors to float type
# so they can be easily input to linear algebra functions
df_best_pos = df_best_pos.applymap(pd.to_numeric)

# Cluster frames with k means to locate the 4 walking passes
frames = df_best_pos.index.values.reshape(-1, 1)
k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

# Sort labels so that the frames are in temporal order
labels = itf.map_sort(k_means.labels_)

# DataFrames for each walking pass in a trial
pass_dfs = nf.group_by_label(df_best_pos, labels)

In [ ]:
df_pass = list(pass_dfs)[0]

In [ ]:
frames = df_pass.index.values

In [ ]:
_, direction_pass = gm.direction_of_pass(df_pass)

# Assign correct sides to feet
df_assigned = asi.assign_sides_pass(df_pass, direction_pass)

## Detect phases

In [ ]:
signal_l = gm.foot_signal(df_assigned.L_FOOT, df_assigned.R_FOOT, direction_pass)

frames_interest = pde.frames_of_interest(signal_l)


In [ ]:
df_phase_l = pde.foot_phases(frames_interest, direction_pass, df_assigned.L_FOOT)
df_phase_r = pde.foot_phases(frames_interest, direction_pass, df_assigned.R_FOOT)

In [ ]:
df_phase_lr = df_phase_l.join(df_phase_r, lsuffix='_L', rsuffix='_R')

In [ ]:
from numpy.linalg import norm
from scipy.spatial.distance import cdist

In [ ]:
def foo(df_stance, column, func):

    series = df_stance.groupby('number')[column].apply(func)

    result_list = [np.median(cdist(a, b)) for a, b in itf.pairwise(series)]

    return np.array(result_list)

In [ ]:
df_stance = df_phase_l[df_phase_l.phase=='stance']

In [ ]:
stride_lengths = foo(df_stance, 'position', np.stack)

In [ ]:
func = compose(nf.to_column, np.stack)
stride_times = foo(df_stance.reset_index(), 'frame', func) / 30

In [ ]:
stride_lengths

In [ ]:
stride_times

In [ ]:
stride_velocities = stride_lengths / stride_times

In [ ]:
stride_velocities

In [ ]:
df_stance_l = df_phase_l[df_phase_l.phase == 'stance']
df_stance_r = df_phase_r[df_phase_r.phase == 'stance']

In [ ]:
column_funcs = {'frame': list, 'position': np.stack}


In [ ]:
df_grouped_l = pf.apply_to_grouped(df_stance_l.reset_index(), 'number', column_funcs)
df_grouped_r = pf.apply_to_grouped(df_stance_r.reset_index(), 'number', column_funcs)

In [ ]:
df_grouped_l['med_frame'] = df_grouped_l.frame.apply(np.median)
df_grouped_r['med_frame'] = df_grouped_r.frame.apply(np.median)

In [ ]:
df_grouped_l.index = df_grouped_l.index.astype('str') + '_L'
df_grouped_r.index = df_grouped_r.index.astype('str') + '_R'

In [ ]:
df = pd.concat([df_grouped_l, df_grouped_r]).sort_values('med_frame').reset_index()

In [ ]:
df